<div id="qe-notebook-header" align="right" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" width="250px" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>

# 求职搜索 III: 拟合值函数迭代

## 目录

- [求职搜索 III: 拟合值函数迭代](#求职搜索-III:-拟合值函数迭代)  
  - [概述](#概述)  
  - [算法](#算法)  
  - [实现](#实现)  
  - [练习](#练习)  

## 概述

在本讲座中，我们再次研究[带有离职的McCall求职搜索模型](https://python.quantecon.org/mccall_model_with_separation.html)，但这次使用连续工资分布。

虽然我们在[第一个求职搜索讲座](https://python.quantecon.org/mccall_model.html)的练习中已经简要考虑过连续工资分布，但在那种情况下，这种改变相对来说是微不足道的。

这是因为我们能够将问题简化为求解单个标量值（持续价值）。

在这里，由于分离，变化不那么简单，因为连续的工资分布导致了不可数的无限状态空间。

无限状态空间带来了额外的挑战，特别是在应用值函数迭代（VFI）时。

这些挑战将促使我们通过添加插值步骤来修改VFI。

VFI和这个插值步骤的组合被称为**拟合值函数迭代**（fitted VFI）。

拟合VFI在实践中非常常见，所以我们将花一些时间来详细研究。

我们将使用以下导入：

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
FONTPATH = "fonts/SourceHanSerifSC-SemiBold.otf"
mpl.font_manager.fontManager.addfont(FONTPATH)
plt.rcParams['font.family'] = ['Source Han Serif SC']

import numpy as np
from numba import jit, float64
from numba.experimental import jitclass

## 算法

该模型与我们[之前学习的](https://python.quantecon.org/mccall_model_with_separation.html)带有工作分离的McCall模型相同，只是工资分布是连续的。

我们将从[简化转换](https://python.quantecon.org/mccall_model_with_separation.html#ast-mcm)后得到的两个Bellman方程开始。

为了适应连续的工资分布，它们采用以下形式：


<a id='equation-bell1mcmc'></a>
$$
d = \int \max \left\{ v(w'), \,  u(c) + \beta d \right\} q(w') d w' \tag{29.1}
$$

和


<a id='equation-bell2mcmc'></a>
$$
v(w) = u(w) + \beta
    \left[
        (1-\alpha)v(w) + \alpha d
    \right] \tag{29.2}
$$

这里的未知量是函数$ v $和标量$ d $。

这些方程与我们之前处理的一对Bellman方程的区别在于：

1. 在[(29.1)](#equation-bell1mcmc)中，原来对有限个工资值的求和变成了对无限集合的积分。  
1. [(29.2)](#equation-bell2mcmc)中的函数$ v $定义在所有$ w \in \mathbb R_+ $上。  


函数 $ q $ 在 [(29.1)](#equation-bell1mcmc) 中是工资分布的密度函数。

其支撑集等于 $ \mathbb R_+ $。

### 值函数迭代

理论上，我们应该按以下步骤进行：

1. 从一个对 [(29.1)](#equation-bell1mcmc)–[(29.2)](#equation-bell2mcmc) 解的猜测值 $ v, d $ 开始。  
1. 将 $ v, d $ 代入 [(29.1)](#equation-bell1mcmc)–[(29.2)](#equation-bell2mcmc) 的右侧，
  计算左侧以获得更新值 $ v', d' $  
1. 除非满足某些停止条件，否则设置 $ (v, d) = (v', d') $
  并返回步骤2。  


然而，在实施这个程序之前，我们必须面对一个问题：
值函数的迭代既不能被精确计算，也不能被存储在计算机中。

要理解这个问题，请考虑 [(29.2)](#equation-bell2mcmc)。

即使 $ v $ 是一个已知函数，存储其更新值 $ v' $ 的唯一方法
是记录其在每个 $ w \in \mathbb R_+ $ 处的值 $ v'(w) $。

显然，这是不可能的。

### 拟合值函数迭代

我们将改用**拟合值函数迭代**。

具体步骤如下：

假设已有当前的猜测值 $ v $。

我们只在有限个”网格”点 $ w_1 < w_2 < \cdots < w_I $ 上记录函数 $ v' $ 的值，然后在需要时根据这些信息重构 $ v' $。

更具体地说，算法将是


<a id='fvi-alg'></a>
1. 从一个数组 $ \mathbf v $ 开始，该数组表示在某些网格点 $ \{w_i\} $ 上的初始值函数猜测值。  
1. 基于 $ \mathbf v $ 和 $ \{w_i\} $，通过插值或近似在状态空间 $ \mathbb R_+ $ 上构建函数 $ v $。  
1. 在每个网格点 $ w_i $ 上获取并记录更新后的函数 $ v'(w_i) $ 的样本。  
1. 除非满足某些停止条件，否则将此作为新数组并返回步骤1。  


我们应该如何处理步骤2？

这是一个函数逼近问题，有很多种方法可以解决。

这里重要的是函数近似方案不仅要对每个$ v $产生良好的近似，而且还要能够很好地配合上述更广泛的迭代算法。

从这两个方面来看，分段线性插值是一个不错的选择。

这种方法

1. 能够很好地配合值函数迭代（参见[[Gordon, 1995](https://python.quantecon.org/zreferences.html#id50)]或[[Stachurski, 2008](https://python.quantecon.org/zreferences.html#id49)]）且  
1. 能保持有用的形状特性，如单调性和凹凸性。  


线性插值将使用[numpy.interp](https://numpy.org/doc/stable/reference/generated/numpy.interp.html)来实现。

下图展示了在网格点$ 0, 0.2, 0.4, 0.6, 0.8, 1 $上对任意函数进行分段线性插值的情况。

In [ ]:
def f(x):
    y1 = 2 * np.cos(6 * x) + np.sin(14 * x)
    return y1 + 2.5

c_grid = np.linspace(0, 1, 6)
f_grid = np.linspace(0, 1, 150)

def Af(x):
    return np.interp(x, c_grid, f(c_grid))

fig, ax = plt.subplots()

ax.plot(f_grid, f(f_grid), 'b-', label='true function')
ax.plot(f_grid, Af(f_grid), 'g-', label='linear approximation')
ax.vlines(c_grid, c_grid * 0, f(c_grid), linestyle='dashed', alpha=0.5)

ax.legend(loc="upper center")

ax.set(xlim=(0, 1), ylim=(0, 6))
plt.show()

## 实现

第一步是为具有分离和连续工资分布的McCall模型构建一个即时编译类。

在本应用中，我们将效用函数设定为对数函数，即$ u(c) = \ln c $。

我们将为工资采用对数正态分布，当$ z $为标准正态分布且$ \mu, \sigma $为参数时，$ w = \exp(\mu + \sigma z) $。

In [ ]:
@jit
def lognormal_draws(n=1000, μ=2.5, σ=0.5, seed=1234):
    np.random.seed(seed)
    z = np.random.randn(n)
    w_draws = np.exp(μ + σ * z)
    return w_draws

这是我们的类。

In [ ]:
mccall_data_continuous = [
    ('c', float64),          # 失业补偿
    ('α', float64),          # 工作分离率
    ('β', float64),          # 折现因子
    ('w_grid', float64[:]),  # 用于拟合VFI的网格点
    ('w_draws', float64[:])  # 用于蒙特卡洛的工资抽样
]

@jitclass(mccall_data_continuous)
class McCallModelContinuous:

    def __init__(self,
                 c=1,
                 α=0.1,
                 β=0.96,
                 grid_min=1e-10,
                 grid_max=5,
                 grid_size=100,
                 w_draws=lognormal_draws()):

        self.c, self.α, self.β = c, α, β

        self.w_grid = np.linspace(grid_min, grid_max, grid_size)
        self.w_draws = w_draws

    def update(self, v, d):

        # 简化名称
        c, α, β = self.c, self.α, self.β
        w = self.w_grid
        u = lambda x: np.log(x)

        # 对数组表示的值函数进行插值
        vf = lambda x: np.interp(x, w, v)

        # 使用蒙特卡洛方法评估积分来更新d
        d_new = np.mean(np.maximum(vf(self.w_draws), u(c) + β * d))

        # 更新v
        v_new = u(w) + β * ((1 - α) * v + α * d)

        return v_new, d_new

然后我们返回当前迭代值作为近似解。

In [ ]:
@jit
def solve_model(mcm, tol=1e-5, max_iter=2000):
    """
    对贝尔曼方程进行迭代直至收敛

    * mcm 是 McCallModel 的一个实例
    """

    v = np.ones_like(mcm.w_grid)    # v的初始猜测值
    d = 1                           # d的初始猜测值
    i = 0
    error = tol + 1

    while error > tol and i < max_iter:
        v_new, d_new = mcm.update(v, d)
        error_1 = np.max(np.abs(v_new - v))
        error_2 = np.abs(d_new - d)
        error = max(error_1, error_2)
        v = v_new
        d = d_new
        i += 1

    return v, d

这是一个函数`compute_reservation_wage`，它接收一个`McCallModelContinuous`实例并返回相应的保留工资。

如果对所有的w都有$ v(w) < h $，那么函数返回np.inf

In [ ]:
@jit
def compute_reservation_wage(mcm):
    """
    通过寻找最小的满足v(w) >= h的w，
    计算McCall模型实例的保留工资。

    如果不存在这样的w，则w_bar被设为np.inf。
    """
    u = lambda x: np.log(x)

    v, d = solve_model(mcm)
    h = u(mcm.c) + mcm.β * d

    w_bar = np.inf
    for i, wage in enumerate(mcm.w_grid):
        if v[i] > h:
            w_bar = wage
            break

    return w_bar

这些练习要求你探索解决方案以及它如何随参数变化。

## 练习

## Exercise 29.1

使用上面的代码探索当工资参数 $ \mu $ 发生变化时，保留工资会发生什么变化。

使用默认参数和 `mu_vals = np.linspace(0.0, 2.0, 15)` 中的 $ \mu $ 值。

保留工资的变化是否如你所预期？

## Solution to[ Exercise 29.1](https://python.quantecon.org/#mfv_ex1)

这是一个解决方案

In [ ]:
mcm = McCallModelContinuous()
mu_vals = np.linspace(0.0, 2.0, 15)
w_bar_vals = np.empty_like(mu_vals)

fig, ax = plt.subplots()

for i, m in enumerate(mu_vals):
    mcm.w_draws = lognormal_draws(μ=m)
    w_bar = compute_reservation_wage(mcm)
    w_bar_vals[i] = w_bar

ax.set(xlabel='mean', ylabel='reservation wage')
ax.plot(mu_vals, w_bar_vals, label=r'$\bar w$ as a function of $\mu$')
ax.legend()

plt.show()

不出所料，当报价分布向右偏移时，求职者更倾向于等待。

## Exercise 29.2

让我们现在来考虑求职者如何对波动性的增加做出反应。

为了理解这一点，请计算当工资报价分布在 $ (m - s, m + s) $ 上均匀分布，且 $ s $ 变化时的保留工资。

这里的想法是我们保持均值不变，但扩大分布范围。

（这是一种*均值保持扩散*。）

使用 `s_vals = np.linspace(1.0, 2.0, 15)` 和 `m = 2.0`。

说明你预期保留工资如何随 $ s $ 变化。

现在计算它。结果是否如你所预期？

## Solution to[ Exercise 29.2](https://python.quantecon.org/#mfv_ex2)

这是一个解决方案

In [ ]:
mcm = McCallModelContinuous()
s_vals = np.linspace(1.0, 2.0, 15)
m = 2.0
w_bar_vals = np.empty_like(s_vals)

fig, ax = plt.subplots()

for i, s in enumerate(s_vals):
    a, b = m - s, m + s
    mcm.w_draws = np.random.uniform(low=a, high=b, size=10_000)
    w_bar = compute_reservation_wage(mcm)
    w_bar_vals[i] = w_bar

ax.set(xlabel='波动性', ylabel='保留工资')
ax.plot(s_vals, w_bar_vals, label=r'工资波动性函数中的$\bar w$')
ax.legend()

plt.show()

保留工资随波动性增加而增加。

人们可能会认为，更高的波动性会使求职者更倾向于接受给定的工作机会，因为接受工作代表确定性，而等待则意味着风险。

但求职就像持有期权：工人只面临上行风险（因为在自由市场中，没有人可以强迫他们接受不好的工作机会）。

更大的波动性意味着更高的上行潜力，这会鼓励求职者继续等待。